In [1]:
!pip -q install langchain faiss-cpu langchain-groq  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [11]:

from bs4 import BeautifulSoup
import requests
import re
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [3]:
groq=""

In [4]:
response = requests.get("https://en.wikipedia.org/wiki/Large_language_model")
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()
text = re.sub(r"\n+", "\n", text)
with open("text_file.txt", "w",encoding='utf-8') as f:
        f.write(text)
f.close()

loader=TextLoader("text_file.txt", encoding="utf-8")
docs=loader.load()
print("The document is: ")
print(docs[0].page_content[:100])
print("The length is: ")
print(len(docs[0].page_content))

The document is: 

Large language model - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
		Navigat
The length is: 
111618


In [5]:
model_name="BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=60)
docs=text_splitter.split_documents(docs)

db=FAISS.from_documents(docs,embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
llm=ChatGroq(groq_api_key=groq, model_name="Llama3-8b-8192")

In [12]:
system_prompt = """
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer. If the answer is not present in the context, simply say 'I don't know the answer' and nothing more.

<context>
{context}
</context>
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question=input("Enter a question: ")

document_chain=create_stuff_documents_chain(llm,prompt)
retriever=db.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)

response=retrieval_chain.invoke({"input":question})
print("------------------------- Answer ---------------------")
print(response["answer"])

output = response["answer"]
output=output.replace("*","\n")

Enter a question: what is the webpage about
------------------------- Answer ---------------------
The webpage appears to be about "Large language models", which is a type of artificial neural network.
